# Milestone 2: GranAda

## 1. Imports

We start by importing all the packages needed

In [1]:
import pandas as pd
import numpy as np
import helper as hp

import matplotlib.pyplot as plt

## 2. Datasets

4 datasets were initially intended to be used. Notes on why the "price" dataset will not be used are later in the notebook.
Download the "All Data" folders for each category in "Bulk Downloads", then place folders in a "__../data/__" folder for this notebook.

* df_prod: http://www.fao.org/faostat/en/#data/QC
* df_value: http://www.fao.org/faostat/en/#data/QV
* df_trade: http://www.fao.org/faostat/en/#data/TM
* df_price: http://www.fao.org/faostat/en/#data/PP
* df_pop: https://population.un.org/wpp/

Note: Population data was extracted from "Estimates" for [1993:2016], tags were manually modified to fit FAO style. Modified CSV file must be given.

## 3. Cleaning the Datasets

We start by identifying common products in our datasets. Many of the definitions of the datasets herunder are taken from the FAO's website, with some trimming and modifications from us.

### 3.1 Loading the data

We start by loading the data into pandas dataframes

In [2]:
df_prod = pd.read_csv("../data/Production_Crops_E_All_Data/Production_Crops_E_All_Data.csv",
                      encoding="unicode_escape")
df_value = pd.read_csv("../data/Value_of_Production_E_All_Data/Value_of_Production_E_All_Data.csv",
                       encoding="unicode_escape")
df_trade = pd.read_csv('../data/Trade_DetailedTradeMatrix_E_All_Data/Trade_DetailedTradeMatrix_E_All_Data.csv',
                       encoding="unicode_escape")
df_price = pd.read_csv("../data/Prices_E_All_Data/Prices_E_All_Data.csv",
                       encoding="unicode_escape")

df_pop = pd.read_csv("../data/Population_table_trimmed.csv",
                     encoding="unicode_escape")
df_pop = df_pop.sort_values(by=['Area'])
df_pop.reset_index(drop=True,inplace=True)

/Users/ENVasquez/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 3.2 Dataset Descriptions

#### Production Dataset

The production dataset contains 180 products (the item row in the dataset) and 258 areas (countries and regions), with the following possible values

* Production Quantity and Seed \[tonnes\] 
* Area harvested \[hectares\]
* Yield \[tonnes per hectare\]

In [5]:
print(f'Number of products: {len(df_prod["Item"].value_counts())}')
print(f'Number of areas: {len(df_prod["Area"].value_counts())}')
df_prod.head()

Number of products: 180
Number of areas: 258


,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2013,Y2013F,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,ha,NaN,NaN,NaN,...,14114.0,NaN,13703.0,NaN,14676.0,NaN,19481.0,NaN,19793.0,NaN
1,2,Afghanistan,221,"Almonds, with shell",5419,Yield,hg/ha,NaN,NaN,NaN,...,29910.0,Fc,19996.0,Fc,16521.0,Fc,16859.0,Fc,13788.0,Fc
2,2,Afghanistan,221,"Almonds, with shell",5510,Production,tonnes,NaN,NaN,NaN,...,42215.0,NaN,27400.0,NaN,24246.0,NaN,32843.0,NaN,27291.0,NaN
3,2,Afghanistan,711,"Anise, badian, fennel, coriander",5312,Area harvested,ha,NaN,M,NaN,...,18500.0,F,30000.0,F,25000.0,F,26019.0,Im,28873.0,Im
4,2,Afghanistan,711,"Anise, badian, fennel, coriander",5419,Yield,hg/ha,NaN,NaN,NaN,...,6757.0,Fc,7167.0,Fc,7200.0,Fc,6923.0,Fc,6830.0,Fc


#### Value Dataset

There are 222 products and 261 areas in this dataset. Value of gross production is provided in both current and constant terms and is expressed in US dollars and Standard Local Currency (SLC). The current value of production measures value in the prices relating to the period being measured. Thus, it represents the market value of food and agricultural products at the time they were produced.

Expressing data series in one uniform currency is useful because it avoids the influence of revaluation in local currency, if any, on value of production. The following elements are present in the dataset.

* Gross Production Value (constant 2004-2006 1000 International USD)
* Gross Production Value (constant 2004-2006 million Standard Local Currency)
* Gross Production Value (constant 2004-2006 million USD)
* Gross Production Value (current million Standard Local Currency)
* Gross Production Value (current million USD)
* Net Production Value (constant 2004-2006 1000 International USD)

In [8]:
print(f'Number of products: {len(df_value["Item"].value_counts())}')
print(f'Number of areas: {len(df_value["Area"].value_counts())}')
df_value.head()

Number of products: 222
Number of areas: 261


,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2012,Y2012F,Y2013,Y2013F,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F
0,2,Afghanistan,221,"Almonds, with shell",152,Gross Production Value (constant 2004-2006 100...,1000 Int. $,NaN,NaN,NaN,...,182960.078000,Fc,124575.156335,Fc,80856.550600,Fc,71549.194374,Fc,96918.674867,Fc
1,2,Afghanistan,221,"Almonds, with shell",154,Net Production Value (constant 2004-2006 1000 I$),1000 Int. $,NaN,NaN,NaN,...,182960.078000,Fc,124575.156335,Fc,80856.550600,Fc,71549.194374,Fc,96918.674867,Fc
2,2,Afghanistan,221,"Almonds, with shell",56,Gross Production Value (current million SLC),SLC,NaN,NaN,NaN,...,14911.000000,Fc,9230.459191,Fc,5563.109132,Fc,4824.773367,Fc,6050.024466,Fc
3,2,Afghanistan,221,"Almonds, with shell",55,Gross Production Value (constant 2004-2006 mil...,SLC,NaN,NaN,NaN,...,6460.468889,Fc,4398.849906,Fc,2855.110444,Fc,2526.460140,Fc,3422.277092,Fc
4,2,Afghanistan,711,"Anise, badian, fennel, coriander",152,Gross Production Value (constant 2004-2006 100...,1000 Int. $,NaN,NaN,NaN,...,69090.200000,Fc,69090.200000,Fc,73114.852434,Fc,79457.968291,Fc,77478.699955,Fc


#### Trade Dataset

Export values are reported as FOB (free on board — that is, the value of the goods plus the value of the services performed to deliver the goods to the border of the exporting country).

Value of agricultural imports: Value of agricultural imports are expressed in thousand US dollars in the FAOSTAT database. Import values are reported as CIF (cost insurance and freight — that is, the value of the goods, plus the value of the services performed to deliver goods to the border of the exporting country, plus the value of the services performed to deliver the good from the border of the exporting country to the border of the importing country).

Quantity of food and agricultural imports: Import quantity represents the physical quantity of the products imported for domestic consumption or processing shipped into a country. It includes re-imports.

In [17]:
print(f'Number of products: {len(df_trade["Item"].value_counts())}')
print(f'Number of areas partner: {len(df_trade["Partner Countries"].value_counts())}')
print(f'Number of areas reporting: {len(df_trade["Reporter Countries"].value_counts())}')
df_trade.head()

Number of products: 424
Number of areas partner: 255
Number of areas reporting: 184


,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item Code,Item,Element Code,Element,Unit,Y1986,...,Y2013,Y2013F,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F
0,2,Afghanistan,4,Algeria,230,"Cashew nuts, shelled",5910,Export Quantity,tonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,*,NaN,NaN
1,2,Afghanistan,4,Algeria,230,"Cashew nuts, shelled",5922,Export Value,1000 US$,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23.0,*,NaN,NaN
2,2,Afghanistan,4,Algeria,1293,Crude materials,5922,Export Value,1000 US$,NaN,...,NaN,NaN,NaN,NaN,1.0,*,1.0,*,5.0,R
3,2,Afghanistan,4,Algeria,561,Raisins,5910,Export Quantity,tonnes,NaN,...,NaN,NaN,12.0,*,NaN,NaN,NaN,NaN,NaN,NaN
4,2,Afghanistan,4,Algeria,561,Raisins,5922,Export Value,1000 US$,NaN,...,NaN,NaN,27.0,*,NaN,NaN,NaN,NaN,NaN,NaN


#### Price Dataset

Note that this dataset is not used because more detailed information on price per tonne can be obtained from the trade dataset.

Producer Prices are prices received by farmers for primary crops as collected at the point of initial sale (prices paid at the farm-gate).

Due to differences in data collection infrastructure and capacity, some countries do vary from this concept by collecting, instead, wholesale or local market prices. While these may be good proxies of farm-gate prices when the marketing chain is very limited, they tend to be poorer proxies in economies where transport and commercial margins constitute a significant share of the final product price. At the far extreme, some countries report retail prices, which are typically very poor proxies for producer prices.

* Producer Price [Local Currency Unit (LCU)/tonnes]
* Producer Price [Standard Local Currency (SLC)/tonnes]
* Producer Price [USD/tonnes]

In [11]:
print(f'Number of products: {len(df_price["Item"].value_counts())}')
print(f'Number of areas: {len(df_price["Area"].value_counts())}')
df_price.head()

Number of products: 212
Number of areas: 179


,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1991,Y1991F,Y1992,...,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F
0,2,Afghanistan,221,"Almonds, with shell",5530,Producer Price (LCU/tonne),LCU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Afghanistan,221,"Almonds, with shell",5531,Producer Price (SLC/tonne),SLC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Afghanistan,515,Apples,5530,Producer Price (LCU/tonne),LCU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Afghanistan,515,Apples,5531,Producer Price (SLC/tonne),SLC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,Afghanistan,526,Apricots,5530,Producer Price (LCU/tonne),LCU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.3 Adjusting Units of Interest

We start by removing rows that aren't in the 1000 USD currency unit from the trade matrix. We only want to keep production values for (area, item) pairs where we have data on the Gross and Net Production value. Two methods to manipulate data were tested. Ultimately using "sets" and redefining the dataframe was much faster.

We separate the value datasets into datasets containing only the net or gross production value.

In [18]:
US_gross = df_value.Element == "Gross Production Value (constant 2004-2006 1000 I$)"
US_net = df_value.Element == "Net Production Value (constant 2004-2006 1000 I$)"

df_value_g = df_value[US_gross]
df_value_n = df_value[US_net]
df_value = df_value[US_net|US_gross]

print(f'Gross Production Value rows: {len(df_value_g.Element)}')
print(f'Net Production Value rows  : {len(df_value_n.Element)}')
print(f'Total Production Value rows: {len(df_value.Element)}')

Gross Production Value rows: 20346
Net Production Value rows  : 20260
Total Production Value rows: 40606


In [19]:
# Obtain all (area, item) pairs for which we have Gross Production Value
AI_value_ind_g = df_value_g['Area'].astype(str) + df_value_g['Item']
AI_value_g = set(AI_value_ind_g.unique())

# Obtain all (area, item) pairs for which we have Net Production Value
AI_value_ind_n = df_value_n['Area'].astype(str) + df_value_n['Item']
AI_value_n = set(AI_value_ind_n.unique())

AI_g = AI_value_g - AI_value_n
AI_n = AI_value_n - AI_value_g

df_value = df_value.drop(AI_value_ind_g[[AI in AI_g for AI in AI_value_ind_g]].index)
df_value.Element.value_counts()

Net Production Value (constant 2004-2006 1000 I$)      20260
Gross Production Value (constant 2004-2006 1000 I$)    20260
Name: Element, dtype: int64

### 3.4 Adjusting Years of Interest

A lot of data is missing for the years before 1993, and the value dataset only contains data up to 2016. Hence we limit or data to the years 1993-2016.

Additionaly, in the values for both "not reported" and "zero" are NaN. The flags in each dataset help specify if the NaN value represents zero or is actually missing.

The reduce the size of the dataframes, a function was written to remove unused columns (such as unused years, and non human readable tags). This function also replaces NaN values by zero when the value is meant to be zero.

In [20]:
def remove_years(df, year_min=[1961,1993], year_max=[2017,2020]):   
    """
    Cleans data matrix by FAO-code columns, and FAO-flag columns. 
    Additionaly removes undesired years from matrices.
    
    :param df: pd.DataFrame The dataframe containing the import, export, production, or value data.
    :param year_min: range of minimum years to be removed [1981,1993], 1993 excluded.
    :param year_min: range of maximum years to be removed [2017,2020], 2020 excluded.
    :return: pd.DataFrame containing trimmed down data.
    """
    #Flags NaN mean "official data". Flag M means missing value. Therefore, [NaN,NaN] in [Y#,Y#F] means zero.
    #Note: for "production value" dataset, Flags NaN is not explicitely reported as the "official data"
    for year in range(year_min[1],year_max[0]):
        yi="Y"+str(year)
        yf="Y"+str(year)+"F"
        df.loc[df[yi].isna() & df[yf].isna(), [yi]] = 0.0
    
    #Keep human readable columns not containign "Code" and "Y&F"
    df = df.drop(columns=[label for label in df.columns if 'Y' and 'F' in label])
    df = df.drop(columns=[label for label in df.columns if 'Code' in label])
    
    #Remove undesired years
    yr_list_min = ["Y"+str(year) for year in range(year_min[0],year_min[1])]
    yr_list_max = ["Y"+str(year) for year in range(year_max[0],year_max[1])]
    df = df.drop(columns=[year for year in df.columns if year in yr_list_min])
    df = df.drop(columns=[year for year in df.columns if year in yr_list_max])
    return df

df_prod = remove_years(df_prod)
df_value = remove_years(df_value)
df_trade = remove_years(df_trade)
df_price = remove_years(df_price)

### 3.5 Adjusting Countries

More countries are present in the country list than number of countries in the world. We noticed that some countries that no longer exist appeared (i.e. "USSR"). These were left in, in case data gaps needed to be explained. To remove specific countries, they can later be removed from the overlap set.

We start by checking, if some countries are partners but never reporters in the trade matrix.

In [21]:
reporter_set = set(df_trade["Reporter Countries"].unique())
partner_set = set(df_trade["Partner Countries"].unique())

print(f" Only in reporter countries:\n {reporter_set.difference(partner_set)} \n")
print(f" Only in partner countries:\n {partner_set.difference(reporter_set)} \n")

 Only in reporter countries:
 set() 

 Only in partner countries:
 {'Liechtenstein', 'Guinea-Bissau', 'Christmas Island', "Democratic People's Republic of Korea", 'Eritrea', 'Niue', 'Wallis and Futuna Islands', 'Andorra', 'Puerto Rico', 'Myanmar', 'Sudan', 'Svalbard and Jan Mayen Islands', 'British Virgin Islands', 'Mozambique', 'Palau', 'Guam', 'Norfolk Island', "Lao People's Democratic Republic", 'Canton and Enderbury Islands', 'Pitcairn Islands', 'Falkland Islands (Malvinas)', 'Liberia', 'Wake Island', 'South Georgia and the South Sandwich Islands', 'Palestine', 'United States Virgin Islands', 'Equatorial Guinea', 'Johnston Island', 'Cayman Islands', 'Western Sahara', 'Haiti', 'American Samoa', 'Czechoslovakia', 'Lesotho', 'US Minor Is.', 'Iraq', 'Angola', 'Mayotte', 'Tokelau', 'Neutral Zone', 'Chad', 'Antarctica', 'Saint Helena, Ascension and Tristan da Cunha', 'Marshall Islands', 'Unspecified Area', 'Northern Mariana Islands', 'Bouvet Island', 'Monaco', 'British Indian Ocean Terri

We can note that many of these "countries" are either small territories where their reporter country would generally the country they are attached to (British Indian Ocean Territory -> United Kingdom), aren't actual countries (Antarctica) or are at war (South Sudan). 

We check how well the different countries line up between the different datasets.

In [22]:
country_p = set(df_prod.Area.unique())
country_v = set(df_value.Area.unique())
country_t = set(df_trade["Partner Countries"].unique())

country_pv = country_p.intersection(country_v)
country_pt = country_p.intersection(country_t)

country_all = country_pv.intersection(country_pt)

print(f"Prod. country list length:      {len(country_p)}")
print(f"Value country list length:      {len(country_v)}")
print(f"PV Joined country list length:  {len(country_pv)}")
print(f"Trade country list length:      {len(country_t)}")
print(f"PT Joined country list length:  {len(country_pt)}\n")
print(f"All Joined country list length: {len(country_all)}")

Prod. country list length:      258
Value country list length:      261
PV Joined country list length:  258
Trade country list length:      255
PT Joined country list length:  221

All Joined country list length: 221


We end up with 221 that are in all datasets. We only keep the countries for which we also have population data.

In [25]:
A_pop_ind = df_pop['Area']
A_pop = set(A_pop_ind.unique())

print("In population matrix only:\n", A_pop - country_all)
print("\nIn production matrix only:\n", country_all - A_pop)

country_all = country_all.intersection(A_pop)

# Verifying no undesired "Territories" remain in list
country_not = set(["World","Asia","Net Food Importing Developing Countries","Americas",\
                 "Low Income Food Deficit Countries","Africa","Land Locked Developing Countries",\
                 "South America","Eastern Asia","Least Developed Countries","Europe","European Union",\
                 "Central America","Southern Europe","Oceania","China","Western Asia","Southern Asia",\
                 "Eastern Africa","Northern Africa","Eastern Europe","South-Eastern Asia","Northern America",\
                 "Small Island Developing States","Western Europe","Central Asia","Western Africa","Middle Africa",\
                 "Caribbean","Southern Africa"])

print(f"\nNone-countries in country list:\n {country_all.intersection(country_not)} \n")

In population matrix only:
 {'Isle of Man', 'Andorra', 'Palau', 'Curaçao', 'State of Palestine', 'Western Africa', 'Bonaire, Sint Eustatius and Saba', 'Channel Islands', 'North Macedonia', 'Mayotte', 'Saint Martin (French part)', 'Northern Mariana Islands', 'Monaco', 'Greenland', 'Turks and Caicos Islands', 'Saint Barthélemy', 'San Marino', 'Gibraltar', 'Sint Maarten (Dutch part)', 'Falkland Islands (Malvinas)', 'Aruba', 'Anguilla', 'Holy See'}

In production matrix only:
 set()

None-countries in country list:
 set() 



We then remove countries that aren't found in all sets.

In [26]:
df_prod = df_prod[[country in country_all for country in df_prod["Area"]]]
df_value = df_value[[country in country_all for country in df_value["Area"]]]
df_trade = df_trade[[country in country_all for country in df_trade["Partner Countries"]]]
df_trade = df_trade[[country in country_all for country in df_trade["Reporter Countries"]]]
df_pop = df_pop[[country in country_all for country in df_pop["Area"]]]

We then verify if any additonal differences between datasets realting to area tags must be resolved.

In [34]:
print(f'Items produced per country:\n{df_prod["Area"].value_counts()}\n\n')
print(f'Items values per country:\n{df_value["Area"].value_counts()}\n\n')

Items produced per country:
China, mainland                                 395
Mexico                                          374
Spain                                           349
Peru                                            331
Turkey                                          322
                                               ... 
Faroe Islands                                     6
United States Virgin Islands                      5
China, Macao SAR                                  3
Saint Helena, Ascension and Tristan da Cunha      2
Liechtenstein                                     2
Name: Area, Length: 213, dtype: int64


Items values per country:
China, mainland                                 310
Mexico                                          280
Spain                                           272
Turkey                                          264
Peru                                            252
                                               ... 
Western Sahara        

The production matrix and value matrix do not have data that correspond to eachother. As such, data without its "production:value" pair is removed.

In [35]:
AI_prod_ind = df_prod['Area'].astype(str) + df_prod['Item']
AI_prod = set(AI_prod_ind.unique())

AI_value_ind = df_value['Area'].astype(str) + df_value['Item']
AI_value = set(AI_value_ind.unique())

AI_all = AI_prod.intersection(AI_value)

print(f"Prod Area/Item pair length:  {len(AI_prod)}")
print(f"Value Area/Item pair length: {len(AI_value)}")
print(f"Area/Item intersect length:  {len(AI_all)}")

df_prod = df_prod[[AI in AI_all for AI in AI_prod_ind]]
df_value = df_value[[AI in AI_all for AI in AI_value_ind]]

Prod Area/Item pair length:  11979
Value Area/Item pair length: 14393
Area/Item intersect length:  10101


In [37]:
print(f'Number of trades as a reporter:\n{df_trade["Reporter Countries"].value_counts()}\n\n')
print(f'Number of trades as a partner:\n{df_trade["Partner Countries"].value_counts()}\n\n')
print(f'Difference\n: {df_trade["Reporter Countries"].value_counts() - df_trade["Partner Countries"].value_counts()}')

Number of trades as a reporter:
Netherlands                 122185
France                      114931
Germany                     110131
United States of America    109423
United Kingdom              102120
                             ...  
Sao Tome and Principe         3208
French Guiana                 3112
Bhutan                        2160
Cook Islands                  1976
Tuvalu                        1556
Name: Reporter Countries, Length: 174, dtype: int64


Number of trades as a partner:
United States of America        122076
France                          113378
Germany                         106747
Netherlands                     104205
United Kingdom                  103736
                                 ...  
South Sudan                       1239
United States Virgin Islands      1163
Liechtenstein                      820
Tokelau                            633
Western Sahara                     177
Name: Partner Countries, Length: 213, dtype: int64


Difference
: Afg

For the trade data, we can note quite a bit of discrepency in the data. Sadly, there isn't much to do about it. As described on the FAO  website, these trades can arise from quite a few scenarios. From ships that sinks midway through a journey, and an export is reported but not an import, to three country trades where different countries report different importing and exporting values.





### 3.6 Adjusting Items

We now want to keep only items present in all datasets.

In [38]:
item_p = set(df_prod.Item.unique())
item_v = set(df_value.Item.unique())
item_t = set(df_trade.Item.unique())
item_pt = item_p.intersection(item_t)
item_all = item_pt.intersection(item_v)

print(f" Production item list length: {len(item_p)}")
print(f" Value item list length:      {len(item_v)}")
print(f" Trade item list length:      {len(item_t)}")
print(f" PT Joined list length:       {len(item_pt)}")
print(f" All item list length:        {len(item_all)}")

#Selecting values that only contain overlap items.
df_prod = df_prod[[item in item_all for item in df_prod["Item"]]]
df_value = df_value[[item in item_all for item in df_value["Item"]]]
df_trade = df_trade[[item in item_all for item in df_trade["Item"]]]

 Production item list length: 161
 Value item list length:      161
 Trade item list length:      423
 PT Joined list length:       120
 All item list length:        120


## 4. Expanding the Datasets

### 4.1 Creating a Cost per Tonne Dataframe

We start by recreating our gross and net production value dataframes.

In [ ]:
df_prod_t = df_prod[df_prod["Element"]=="Production"]
df_prod_t.reset_index(drop=True,inplace=True)

df_value_g = df_value[df_value["Element"]=="Gross Production Value (constant 2004-2006 1000 I$)"]
df_value_g.reset_index(drop=True,inplace=True)

df_value_n = df_value[df_value["Element"]=="Net Production Value (constant 2004-2006 1000 I$)"]
df_value_n.reset_index(drop=True,inplace=True)

We then create a total cost matrix by subtracting net production value from gross production value.

In [ ]:
df_cost_numbers = df_value_g.loc[:,"Y1993":"Y2016"].subtract(df_value_n.loc[:,"Y1993":"Y2016"])

df_cost_data = df_value_g.loc[:,:"Unit"]
df_cost_data["Unit"]="1000 US$"
df_cost_data["Element"]="Gross Production Value minus Net Production Value"

df_cost = pd.concat([df_cost_data, df_cost_numbers], axis=1, sort=False)
df_cost.head()

To compute the cost per tonne, we divide the cost matrix by the production quantity of that item.

In [ ]:
df_costpt_numbers = df_cost.loc[:,"Y1993":"Y2016"].divide(df_prod_t.loc[:,"Y1993":"Y2016"])
df_costpt_numbers[df_costpt_numbers == np.Inf] = np.NaN

df_costpt_data = df_cost.loc[:,:"Unit"]
df_costpt_data["Unit"]="1000 US$/tonne"
df_costpt_data["Element"]="Cost per tonne"

df_costpt = pd.concat([df_costpt_data, df_costpt_numbers], axis=1, sort=False)
df_costpt.head()

w